In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
# visualization

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# 소수점 둘째자리까지 표시하도록 설정 (원하는 자릿수로 변경 가능)
pd.options.display.float_format = '{:.2f}'.format

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import eli5
from eli5.sklearn import PermutationImportance

## Optuna tunning for XGB
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import optuna

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(
fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
name='NanumBarunGothic') # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe) # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns


## Optuna visulization
import plotly.express as px
import plotly.graph_objects as go
import plotly

# 열의 개수를 출력할 때 모두 표시하도록 설정
pd.set_option('display.max_columns', None)

In [29]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = './data/train.csv'
test_path = './data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
submission = pd.read_csv("./data/sample_submission.csv")

In [30]:
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in train.columns:
    if pd.api.types.is_numeric_dtype(train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '해제사유발생일', '단지소개기존clob', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수', '좌표X', '좌표Y', 'target']
범주형 변수: ['시군구', '번지', '아파트명', '도로명', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


In [31]:
# define columnTransformer for standardizing num_cols and encoding cat_cols
preprocessor = ColumnTransformer(
    [  # ('num', StandardScaler(), num_cols),
        ('cat', OrdinalEncoder(), categorical_columns_v2),
    ])

In [38]:
all = pd.concat([train.drop(['target'], axis=1), test], axis=0)

In [40]:
oe_dict = {}
for col in all.columns:
    oe_dict[col] = OrdinalEncoder()
    print(f'{col} : {all[col].unique()}')
    oe_dict[col] = oe_dict[col].fit(all[col].astype(str).values.reshape(-1, 1))

시군구 : ['서울특별시 강남구 개포동' '서울특별시 강남구 논현동' '서울특별시 강남구 대치동' '서울특별시 강남구 도곡동'
 '서울특별시 강남구 삼성동' '서울특별시 강남구 세곡동' '서울특별시 강남구 수서동' '서울특별시 강남구 신사동'
 '서울특별시 강남구 역삼동' '서울특별시 강남구 압구정동' '서울특별시 강남구 일원동' '서울특별시 강남구 자곡동'
 '서울특별시 강남구 청담동' '서울특별시 강동구 강일동' '서울특별시 강남구 율현동' '서울특별시 강동구 고덕동'
 '서울특별시 강동구 길동' '서울특별시 강동구 둔촌동' '서울특별시 강동구 명일동' '서울특별시 강동구 상일동'
 '서울특별시 강동구 성내동' '서울특별시 강동구 암사동' '서울특별시 강동구 천호동' '서울특별시 강북구 미아동'
 '서울특별시 강북구 번동' '서울특별시 강서구 가양동' '서울특별시 강북구 수유동' '서울특별시 강북구 우이동'
 '서울특별시 강서구 내발산동' '서울특별시 강서구 등촌동' '서울특별시 강서구 공항동' '서울특별시 강서구 마곡동'
 '서울특별시 강서구 방화동' '서울특별시 강서구 염창동' '서울특별시 강서구 화곡동' '서울특별시 관악구 남현동'
 '서울특별시 관악구 봉천동' '서울특별시 관악구 신림동' '서울특별시 광진구 광장동' '서울특별시 광진구 구의동'
 '서울특별시 광진구 자양동' '서울특별시 광진구 군자동' '서울특별시 광진구 화양동' '서울특별시 구로구 개봉동'
 '서울특별시 구로구 고척동' '서울특별시 광진구 중곡동' '서울특별시 구로구 가리봉동' '서울특별시 구로구 구로동'
 '서울특별시 구로구 오류동' '서울특별시 구로구 천왕동' '서울특별시 금천구 독산동' '서울특별시 구로구 온수동'
 '서울특별시 구로구 궁동' '서울특별시 구로구 신도림동' '서울특별시 구로구 항동' '서울특별시 금천구 가산동'
 '서울특별시 금천구 시흥동' '서울특별시 노원구 공릉동' '서울특별시 노원구 상계동' '서울특별시 노원구 월계동'
 '서울특별시 노원구 중계동' '서

In [41]:
for col in categorical_columns_v2:
    # train[col] = lbl.fit_transform(train[col])
    # test[col] = lbl.transform(test[col])
    train[col] = oe_dict[col].transform(train[col].astype(str).values.reshape(-1, 1))
    test[col] = oe_dict[col].transform(test[col].astype(str).values.reshape(-1, 1))

In [42]:
# Target과 독립변수들을 분리해줍니다.
y_train = train['target']
X_train = train.drop(['target'], axis=1)
# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [43]:
def objective(trial):


    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 750),
        'max_depth': trial.suggest_int('max_depth', 3, 50),
        'min_child_weight': trial.suggest_float('min_child_weight', 2, 50),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.2, 1),
        'gamma': trial.suggest_float('gamma', 1e-4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.2, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.2, 1)
    }
    # KFold : 회귀모델 / StratifiedKFold : 분류모델
    # kf = KFold(n_splits = 5, shuffle = True, random_state = 1996)  
    
    xgbmodel_optuna = XGBRegressor(**params, random_state=1996, tree_method='gpu_hist')
    # xgbmodel_optuna = XGBClassifier(**params, random_state = 1996)
    
    cv = np.sqrt(-cross_val_score(xgbmodel_optuna, X_train, y_train,
                                  scoring='neg_mean_squared_error', cv=KFold(n_splits=5))).mean()
    return cv

In [44]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2024-01-25 11:27:18,095] A new study created in memory with name: no-name-965e80e9-5212-40fd-9c00-48fe039f04ec
[W 2024-01-25 11:32:25,120] Trial 0 failed with parameters: {'n_estimators': 156, 'max_depth': 40, 'min_child_weight': 6.790745293093803, 'learning_rate': 0.063196872992415, 'subsample': 0.5306757296491682, 'gamma': 0.8758145909827089, 'colsample_bytree': 0.7475289606445621, 'colsample_bylevel': 0.3179117074845791, 'colsample_bynode': 0.5545260051976612} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\demya\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\demya\AppData\Local\Temp\ipykernel_2336\1554051499.py", line 21, in objective
    cv = np.sqrt(-cross_val_score(xgbmodel_optuna, X_train, y_train,
  File "C:\Users\demya\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 509, in cross_val_score
    cv_results = cr

KeyboardInterrupt: 

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

# Hyperparameter Importances를 통해서 parameter를 고정시켜라.
# 그리고 나머지 것들을 진행시켜라.
optuna.visualization.plot_param_importances(study)

optuna.visualization.plot_optimization_history(study)

# model finalization# optuna는 hyperparameter를 조정.
best_params = study.best_params
best_model = XGBRegressor(**best_params,
                          random_state=1996,
                          tree_method='gpu_hist',
                          )

pred = best_model.fit(X_train, y_train)

print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, best_model.predict(X_val)))}')

In [4]:
# model finalization
# optuna는 hyperparameter를 조정.

best_params = study.best_params
best_model = XGBRegressor(**best_params,
  random_state=1996,
  tree_method = 'gpu_hist',
)

pred = best_model.fit(X,y,verbose=False).predict(X_test)

submission = pd.DataFrame(pred.astype(int),columns=['target'])
submission.to_csv('sub.csv',index=False)

연속형 변수: ['본번', '부번', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '해제사유발생일', '단지소개기존clob', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수', '좌표X', '좌표Y', 'target']
범주형 변수: ['시군구', '번지', '아파트명', '도로명', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


NameError: name 'train1' is not defined